## Extracting features using neural networks

Pass each window 10 times and get the last layer parameters
Each channel will have 10 features

In [2]:
#Import necessary libraries

import pandas as pd
import numpy as np
from scipy import signal
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle as pkl
import itertools 
import glob
from sklearn import svm 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, f1_score
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

%matplotlib inline 
# %matplotlib qt


In [3]:
# Import raw datasets, non-windowed 
think_df = pd.read_pickle("F:\EEG-data\\think-count\\raw/think.pkl")
count_df = pd.read_pickle("F:\EEG-data\\think-count\\raw/think.pkl")

print(think_df.shape)
print(count_df.shape)
print(think_df.iloc[0,0]) # Each element in dataframe is a single timestep
print(count_df.iloc[0,0])

(3840, 32)
(3840, 32)
26.553326000000002
26.553326000000002


In [4]:
# # Testing on a single column 
testcol = think_df.iloc[:,0].values

# # Normalize column 
testcol = keras.utils.normalize(testcol).flatten()

time = np.linspace(0,10,len(testcol))
split_time = int(len(testcol)*0.8)
time_train = time[:split_time]
x_train = testcol[:split_time]
time_valid = time[split_time:]
x_valid = testcol[split_time:]

window_size = 12
# batch_size = 32
# shuffle_buffer_size = 1000

def windowed_dataset(dataframe,window_size):
    N,M = dataframe.shape
    ds_labels = np.zeros((N-window_size+1,1,M))
    ds_windows = np.zeros((N-window_size+1,window_size,M))
    for i in range(M):
        channel = dataframe.iloc[:,i].values
        channel = keras.utils.normalize(channel).flatten()
        dataset = tf.data.Dataset.from_tensor_slices(channel)
        win_dataset = dataset.window(window_size+1,1,drop_remainder=True)
        win_dataset = win_dataset.flat_map(lambda window: window.batch(window_size+1))
        split_dataset = win_dataset.map(lambda window: (window[:-1],window[-1]))

        labels = np.zeros((N-window_size+1,1))
        newcol = np.zeros((N-window_size+1,window_size))
        for j,win in enumerate(split_dataset):
            newcol[j,:] = win[0].numpy()
            labels[j,:] = win[1].numpy()

        ds_labels[:,:,i] = labels
        ds_windows[:,:,i] = newcol

    return ds_windows, ds_labels

# dataset = tf.data.Dataset.from_tensor_slices(testcol)
# win_dataset = dataset.window(window_size+1,1,drop_remainder=True)
# win_dataset = win_dataset.flat_map(lambda window: window.batch(window_size+1))
# split_dataset = win_dataset.map(lambda window: (window[:-1],window[-1]))

# newcol = np.zeros((3829,12))
# labels = []
# for i,win in enumerate(win_dataset):
#     newcol[i,:] = list(win.as_numpy_iterator())

# Don't need to shuffle 

# def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
#   dataset = tf.data.Dataset.from_tensor_slices(series)
#   dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
#   dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
#   dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
#   dataset = dataset.batch(batch_size).prefetch(1)
#   return dataset




In [ ]:
# Shape of windowed data: 3829,12,32; shape of labels: 3829,1,32
# Shape of features: 3829,10,32
# N = no. of examples, M = no. of channels, 
# Shape of windowed data: N-window_size+1,window_size,M
# Shape of labels: N-window_size+1,1,M
# Shape of features: N-window_size=1,10,M

In [5]:
think_xtrainset,think_ytrainset = windowed_dataset(think_df,12)
count_xtrainset,count_ytrainset = windowed_dataset(count_df,12)

print(think_xtrainset.shape)
print(think_ytrainset.shape)
print(count_xtrainset.shape)
print(count_ytrainset.shape)

(3829, 12, 32)
(3829, 1, 32)
(3829, 12, 32)
(3829, 1, 32)


In [6]:
np.save("F:\EEG-data\\think-count\\raw\windows/think_xtrain",think_xtrainset)
np.save("F:\EEG-data\\think-count\\raw\windows/think_ytrain",think_ytrainset)
np.save("F:\EEG-data\\think-count\\raw\windows/count_xtrain",count_xtrainset)
np.save("F:\EEG-data\\think-count\\raw\windows/count_ytrain",count_ytrainset)


In [69]:
lastDense = tf.keras.layers.Dense(1, activation="relu")
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape = [window_size], activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"), 
    lastDense
])
model.compile(loss=tf.keras.losses.Huber(),
              optimizer='adam',
              metrics=["mae"])
# model.compile(loss='mae',optimizer='adam')
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 10)                130       
_________________________________________________________________
dense_67 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_65 (Dense)             (None, 1)                 11        
Total params: 251
Trainable params: 251
Non-trainable params: 0
_________________________________________________________________


In [73]:
tf.random.set_seed(10)
N,_,M = think_xtrainset.shape
ds_weights = np.zeros((N,10,M))

for i in range(N): #N
    for j in range(M):#M
        train_set = tf.data.Dataset.from_tensor_slices((think_xtrainset[i,:,j].reshape(1,12), think_ytrainset[i,:,j].reshape(1,1)))        
        train_set = train_set.batch(1)
        model.fit(train_set,epochs=10,verbose=0)
        weights = (lastDense.get_weights()[0]).flatten()
        ds_weights[i,:,j] = weights

In [74]:
print(ds_weights[10,:,1])

[-0.12320234  0.07362305 -0.0142203   0.7342698   0.03959388 -0.00679281
  0.38223428 -0.53269953 -0.40846771  0.71088701]


In [70]:
# N,_,M = think_xtrainset.shape
train_set = tf.data.Dataset.from_tensor_slices((think_xtrainset[1,:,1].reshape(1,12), think_ytrainset[1,:,1].reshape(1,1)))
train_set = train_set.batch(1).repeat()
history = model.fit(train_set,epochs=10,verbose=0,steps_per_epoch=100)
x = think_xtrainset[1,:,1].reshape(1,12)
y = think_ytrainset[1,:,1].reshape(1,1)
ypred = model.predict(x).flatten()
print(ypred)
# time = np.linspace(1,10,13)
# plt.plot(time[:-1],x.flatten())
# plt.plot(time[-1],y,'x')
# plt.plot(time[-1],ypred,'o')

[0.]
